In [1]:
import numpy as np
import pandas as pd
import pickle
import urllib3
import re
from datetime import datetime, timedelta
from collections import OrderedDict

In [2]:
dates = ["2008-01", "2019-10"]
start, end = [datetime.strptime(_, '%Y-%m') for _ in dates]
month_ordereddict = OrderedDict(((start + timedelta(_)).strftime("%Y%m") + ".txt", None) for _ in range((end - start).days)).keys()
month_list = list(month_ordereddict)

In [3]:
url_base = "https://www.census.gov/construction/bps/txt/tb2u"

In [4]:
data_list = []
for month in month_list:
    http = urllib3.PoolManager()
    url = f'{url_base}{month}'
    r = http.request('GET', url)
    data = r.data.decode('utf-8')
    data_split = re.split("\n", data)
    louisiana_full_string = [x for x in data_split if "Louisiana" in x][0]
    louisiana_split_date = louisiana_full_string.split()
    louisiana_split_date.append(month[0:4]+ "-" + month[4:6])
    data_list.append(louisiana_split_date)

In [5]:
data_list.insert(0, ['State', 'permit_total', '1 Unit', '2 Units', '3 Units', '3 and 4 Units', '5 Units or More', 'dtime'])

In [6]:
html_permits = pd.DataFrame(data_list[1::], columns= data_list[0])

In [7]:
html_permits.drop(columns=['State','1 Unit', '2 Units', '3 Units', '3 and 4 Units', '5 Units or More'], inplace = True)

In [8]:
html_permits['dtime'] = pd.to_datetime(html_permits['dtime'])

In [9]:
html_permits

,permit_total,dtime
0,1561,2008-01-01
1,1311,2008-02-01
2,1494,2008-03-01
3,2094,2008-04-01
4,1560,2008-05-01
...,...,...
136,1303,2019-05-01
137,1258,2019-06-01
138,1454,2019-07-01
139,1497,2019-08-01


In [10]:
excel_permits = pd.read_excel('permits_2019_11_2020_11.xlsx')

In [11]:
excel_permits['dtime'] = pd.to_datetime(excel_permits['dtime'])

In [12]:
permits = pd.concat([html_permits, excel_permits])

In [14]:
permits.permit_total = permits.permit_total.astype(int)
permits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 154 entries, 0 to 12
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   permit_total  154 non-null    int64         
 1   dtime         154 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1)
memory usage: 3.6 KB


In [15]:
permits.to_pickle("permits.pkl")